In [11]:
module Symbolic

abstract type AbstractSymbolic end

struct SymbolicVal <: AbstractSymbolic
    val
end

struct SymbolicVar <: AbstractSymbolic
    var::Symbol
end

struct SymbolicExpr <: AbstractSymbolic
    head::Symbol
    args::Vector{AbstractSymbolic}
end

"""
Operations
"""

const operations = [:+, :-, :*, :/, :^, :exp, :sqrt, :log]

for op = operations
    @eval function Base.$op(x::T, y::S) where {T <: AbstractSymbolic, S <: Number}
        SymbolicExpr($(Expr(:quote, op)), [x, SymbolicVal(y)])
    end
end

for op = operations
    @eval function Base.$op(x::T, y::S) where {T <: Number, S <: AbstractSymbolic}
        SymbolicExpr($(Expr(:quote, op)), [SymbolicVal(x), y])
    end
end

for op = operations
    @eval function Base.$op(x::T, y::S) where {T <: AbstractSymbolic, S <: AbstractSymbolic}
        SymbolicExpr($(Expr(:quote, op)), [x, y])
    end
end

function _expr(x::SymbolicExpr)
    args = [_expr(u) for u = x.args]
    Expr(:call, x.head, args...)
end

function _expr(x::SymbolicVar)
    x.var
end

function _expr(x::SymbolicVal)
    x.val
end

macro param(x)
    if typeof(x) == Symbol
        :(SymbolicVar($(Expr(:quote, x))))
    else
        x
    end
end

end

Main.Symbolic

In [13]:
Symbolic.@param x

Main.Symbolic.SymbolicVar(:x)

In [135]:
x = Symbolic.SymbolicVal(:x)

Main.Symbolic.SymbolicVal(:x)

In [136]:
y = 10

10

In [137]:
z = x * y

Main.Symbolic.SymbolicExpr(:*, Main.Symbolic.AbstractSymbolic[Main.Symbolic.SymbolicVal(:x), Main.Symbolic.SymbolicVal(10)])

In [138]:
f = z + 10

Main.Symbolic.SymbolicExpr(:+, Main.Symbolic.AbstractSymbolic[Main.Symbolic.SymbolicExpr(:*, Main.Symbolic.AbstractSymbolic[Main.Symbolic.SymbolicVal(:x), Main.Symbolic.SymbolicVal(10)]), Main.Symbolic.SymbolicVal(10)])

In [139]:
Symbolic._expr(f)

:(x * 10 + 10)

In [14]:
function tes(; kwargs...)
    println(kwargs)
end

tes (generic function with 1 method)

In [16]:
tes(a=0, b=1)

Base.Pairs(:a => 0, :b => 1)
